In [15]:
!pip install folium

  Using cached folium-0.18.0-py2.py3-none-any.whl (108 kB)
  Using cached branca-0.8.1-py3-none-any.whl (26 kB)
  Using cached jinja2-3.1.6-py3-none-any.whl (134 kB)
  Attempting uninstall: jinja2
    Found existing installation: Jinja2 2.11.3
    Uninstalling Jinja2-2.11.3:
      Successfully uninstalled Jinja2-2.11.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
anaconda-project 0.9.1 requires ruamel-yaml, which is not installed.
sphinx 4.0.1 requires Jinja2<3.0,>=2.3, but you have jinja2 3.1.6 which is incompatible.
sphinx 4.0.1 requires MarkupSafe<2.0, but you have markupsafe 2.1.5 which is incompatible.

[notice] A new release of pip available: 22.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import requests
import time
import pandas as pd
import folium
from folium.plugins import MarkerCluster
import json

In [3]:
API_KEY = 'nuh uh'  
HEADERS = {'Authorization': f'Bearer {API_KEY}'}
SEARCH_ENDPOINT = 'https://api.yelp.com/v3/businesses/search'

In [4]:
# search for all major restaurants within all postal codes in richmond
categories = ['chinese', 'japanese', 'korean', 'vietnamese', 'indian', 'mexican', 'thai', 'italian', 'pizza', 'burgers']
postal_codes = ['V6V', 'V6W', 'V6X', 'V6Y', 'V7A', 'V7C', 'V7E']

all_results = []

# way to by pass post request restaurant limit (50)
# request restaurants that fall within category in specific postal code
for category in categories:
    for postal in postal_codes:
        print(f"Searching: {category} in {postal}...")
        offset = 0
        while offset <= 190: 
            params = {
                'location': f'{postal}, Richmond, BC',
                'categories': f'{category},restaurants',
                'limit': 50,
                'offset': offset
            }
            
            # make request + error handling
            response = requests.get(SEARCH_ENDPOINT, headers=HEADERS, params=params)
            if response.status_code != 200:
                print(f"Error {response.status_code}: {response.text}")
                break

            data = response.json()
            businesses = data.get('businesses', [])
            if not businesses:
                break

            # add to array
            for biz in businesses:
                all_results.append({
                    'id': biz['id'],
                    'name': biz['name'],
                    'address': ", ".join(biz['location']['display_address']),
                    'latitude': biz['coordinates']['latitude'],
                    'longitude': biz['coordinates']['longitude'],
                    'categories': ", ".join([cat['title'] for cat in biz.get('categories', [])])
                })

            offset += 50
            time.sleep(0.5)

# drop duplicates
df = pd.DataFrame(all_results)
df.drop_duplicates(subset='id', inplace=True)
df.reset_index(drop=True, inplace=True)
df

Searching: chinese in V6V...
Searching: chinese in V6W...
Searching: chinese in V6X...
Searching: chinese in V6Y...
Searching: chinese in V7A...
Searching: chinese in V7C...
Searching: chinese in V7E...
Searching: japanese in V6V...
Searching: japanese in V6W...
Searching: japanese in V6X...
Searching: japanese in V6Y...
Searching: japanese in V7A...
Searching: japanese in V7C...
Searching: japanese in V7E...
Searching: korean in V6V...
Searching: korean in V6W...
Searching: korean in V6X...
Searching: korean in V6Y...
Searching: korean in V7A...
Searching: korean in V7C...
Searching: korean in V7E...
Searching: vietnamese in V6V...
Searching: vietnamese in V6W...
Searching: vietnamese in V6X...
Searching: vietnamese in V6Y...
Searching: vietnamese in V7A...
Searching: vietnamese in V7C...
Searching: vietnamese in V7E...
Searching: indian in V6V...
Searching: indian in V6W...
Searching: indian in V6X...
Searching: indian in V6Y...
Searching: indian in V7A...
Searching: indian in V7C...

,id,name,address,latitude,longitude,categories
0,y7_33jucQRo37eQEshPeMQ,Shoestring Cafe,"4611 No 6 Road, Suite 180, Richmond, BC V6V 2L...",49.179874,-123.069558,"French, Seafood, Steakhouses"
1,pgyPhwk3lB1DxedDzygUUg,The Fish Man,"1170-8391 Alexandra Road, Richmond, BC V6X 1C3...",49.178466,-123.130252,"Seafood, Chinese, Barbeque"
2,crM1idgY_glhtxXT5kERNg,Kingyo,"871 Denman Street, Vancouver, BC V6G 2L9, Canada",49.290739,-123.137050,"Japanese, Tapas/Small Plates"
3,GmEeIcJ9vR04kxzYgPMEgg,Guu with Garlic,"1698 Robson Street, Vancouver, BC V6G 1C7, Canada",49.290249,-123.133814,"Japanese, Tapas/Small Plates"
4,4EV_ZcQmjAmP3pmO-_nb2A,Miku,"200 Granville Street, Suite 70, Vancouver, BC ...",49.286913,-123.113049,"Japanese, Sushi Bars"
...,...,...,...,...,...,...
483,vKIpb6S6C3Cc4GvDorBn2w,Moncton Pizzeria,"3680 Moncton Street, Richmond, BC V7E 3A4, Canada",49.125025,-123.183609,Pizza
484,W3fHTFr9TE6OwP2AOTRdhA,Kuroki Kaiseki Omakase Kitchen,"110-3880 Chatham Street, Richmond, BC V7E 2Z5,...",49.126214,-123.182128,"Japanese, Sushi Bars"
485,CHYGStOcv5MgdIwBpSDgSg,John's Pan Pizza,"12420 No 1 Road, Suite 130, Richmond, BC V7E 6...",49.124770,-123.180870,"Pizza, Salad, Italian"
486,WGA6x5gJ7yum9kV8SSgrvw,Waves Coffee House,"120 - 12231 First Avenue, Richmond, BC V7E 3M3...",49.124180,-123.182890,"Coffee & Tea, Cafes"


In [5]:
# manually add Domino's and Torake ---
params_list = [
    {'term': "Domino's Pizza", 'location': 'Richmond, BC'},
    {'term': "Torake", 'location': 'Richmond, BC'}
]

for params in params_list:
    params['limit'] = 1
    response = requests.get(SEARCH_ENDPOINT, headers=HEADERS, params=params)
    biz = response.json().get('businesses', [None])[0]
    if not biz or biz['id'] in df['id'].values:
        continue

    coords = biz['coordinates']
    df.loc[len(df)] = {
        'id': biz['id'],
        'name': biz['name'],
        'latitude': coords['latitude'],
        'longitude': coords['longitude'],
        'categories': ", ".join([cat['title'] for cat in biz.get('categories', [])]),
        'address': ", ".join(biz['location']['display_address']),
    }

df.reset_index(drop=True, inplace=True)
print(f"Final total restaurants: {len(df)}")
df

Final total restaurants: 490


,id,name,address,latitude,longitude,categories
0,y7_33jucQRo37eQEshPeMQ,Shoestring Cafe,"4611 No 6 Road, Suite 180, Richmond, BC V6V 2L...",49.179874,-123.069558,"French, Seafood, Steakhouses"
1,pgyPhwk3lB1DxedDzygUUg,The Fish Man,"1170-8391 Alexandra Road, Richmond, BC V6X 1C3...",49.178466,-123.130252,"Seafood, Chinese, Barbeque"
2,crM1idgY_glhtxXT5kERNg,Kingyo,"871 Denman Street, Vancouver, BC V6G 2L9, Canada",49.290739,-123.137050,"Japanese, Tapas/Small Plates"
3,GmEeIcJ9vR04kxzYgPMEgg,Guu with Garlic,"1698 Robson Street, Vancouver, BC V6G 1C7, Canada",49.290249,-123.133814,"Japanese, Tapas/Small Plates"
4,4EV_ZcQmjAmP3pmO-_nb2A,Miku,"200 Granville Street, Suite 70, Vancouver, BC ...",49.286913,-123.113049,"Japanese, Sushi Bars"
...,...,...,...,...,...,...
485,CHYGStOcv5MgdIwBpSDgSg,John's Pan Pizza,"12420 No 1 Road, Suite 130, Richmond, BC V7E 6...",49.124770,-123.180870,"Pizza, Salad, Italian"
486,WGA6x5gJ7yum9kV8SSgrvw,Waves Coffee House,"120 - 12231 First Avenue, Richmond, BC V7E 3M3...",49.124180,-123.182890,"Coffee & Tea, Cafes"
487,evMi_871kCorAYHtpXOXvg,Steveston Built Local Taphouse & Grill,"130-12480 No 1Road, Richmond, BC V7E, Canada",49.124290,-123.181065,"Bars, Burgers"
488,v6GRZxd-96gojJrXKFcLdw,Domino's Pizza,"9471 No. 2 Road, Unit 120, Richmond, BC V7E 2C...",49.144227,-123.159686,"Pizza, Chicken Wings, Sandwiches"


In [6]:
# save yelp restaurants data so i dont need to access api again
df.to_csv("yelp_restaurants_richmond.csv", index=False)
# load it after saving
df = pd.read_csv("yelp_restaurants_richmond.csv")

## Create Context Map

In [7]:
# m = folium.Map(location=[49.1666, -123.1336], zoom_start=13, tiles='CartoDB dark_matter')

# for _, row in df.iterrows():
#     folium.CircleMarker(
#         location=[row['latitude'], row['longitude']],
#         radius=1.5,
#         color='#00ffff',
#         fill=True,
#         fill_opacity=1,
#         stroke=False
#     ).add_to(m)

# m.save("richmond_restaurants_map.html")
# print("🌐 Map saved as richmond_restaurants_map.html")

In [8]:
# create your base map focused on richmond
m = folium.Map(location=[49.1666, -123.1336], zoom_start=13, tiles='CartoDB dark_matter')

# add all restauarants
for _, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=1.5,
        color='#00ffff',
        fill=True,
        fill_opacity=1,
        stroke=False
    ).add_to(m)

# made torake red mark
folium.CircleMarker(
    location=[49.144259, -123.159658],
    radius=5,
    color='red',
    fill=True,
    fill_opacity=1,
    tooltip='Torake'
).add_to(m)    

# Load GeoJSON with bus stops
with open("stops.geojson") as f:
    bus_stop_data = json.load(f)

# add bus stops
for feature in bus_stop_data['features']:
    geom_type = feature['geometry']['type']
    coords = feature['geometry']['coordinates']

    if geom_type == 'Point':
        lon, lat = coords
        folium.CircleMarker(
            location=[lat, lon],
            radius=3,
            color='green',
            fill=True,
            fill_opacity=1,
            stroke=False,
            tooltip=feature['properties'].get('stop_name', 'Bus Stop')
        ).add_to(m)
        
# add legend
legend_html = '''
<div style="position: fixed; 
     bottom: 30px; left: 30px; width: 180px; height: 110px; 
     background-color: white; border:2px solid grey; z-index:9999; font-size:14px;
     padding: 10px;">
     <b>Legend</b><br>
     <i style="background: blue; width: 10px; height: 10px; float: left; margin-right: 5px; border-radius: 50%; display: inline-block;"></i> Restaurants<br>
     <i style="background: green; width: 10px; height: 10px; float: left; margin-right: 5px; border-radius: 50%; display: inline-block;"></i> Bus Stops<br>
     <i style="background: red; width: 10px; height: 10px; float: left; margin-right: 5px; border-radius: 50%; display: inline-block;"></i> Torake
</div>
'''

m.get_root().html.add_child(folium.Element(legend_html))

# save
m.save("richmond_restaurants_bus_stops.html")
m.save("context map.html")
print ("updated map")


updated map


In [9]:
# Filter rows where "Japanese" appears in the categories
japanese_restaurants = df[df['categories'].str.contains('Japanese', case=False, na=False)]

# View the result
print(f"Found {len(japanese_restaurants)} Japanese restaurants.")
japanese_restaurants.head()


Found 75 Japanese restaurants.


,id,name,address,latitude,longitude,categories
2,crM1idgY_glhtxXT5kERNg,Kingyo,"871 Denman Street, Vancouver, BC V6G 2L9, Canada",49.290739,-123.137050,"Japanese, Tapas/Small Plates"
3,GmEeIcJ9vR04kxzYgPMEgg,Guu with Garlic,"1698 Robson Street, Vancouver, BC V6G 1C7, Canada",49.290249,-123.133814,"Japanese, Tapas/Small Plates"
4,4EV_ZcQmjAmP3pmO-_nb2A,Miku,"200 Granville Street, Suite 70, Vancouver, BC ...",49.286913,-123.113049,"Japanese, Sushi Bars"
16,lk1xyEwVmB8HIYOiKbkUIw,Nobi Nobi,"11320 Steveston Hwy, Unit 110, Richmond, BC V7...",49.133102,-123.098042,Japanese
35,iid0VenH7rIp3AWuyQAAHw,Saku,"1588 Robson Street, Vancouver, BC V6G 2G5, Canada",49.289223,-123.132266,Japanese
